In [1]:
import torch.nn as nn 
from models import create_model 
import yaml 
from omegaconf import OmegaConf
from arguments import jupyter_parser
import torch 
import torch.nn.functional as F 

import logging
import wandb
import time
import os
import json

import numpy as np
import pandas as pd

import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from collections import OrderedDict
from accelerate import Accelerator

from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, \
                            balanced_accuracy_score, classification_report, confusion_matrix, accuracy_score

from query_strategies import create_query_strategy, create_labeled_index
from models import ResNetSimCLR
from utils import NoIndent, MyEncoder
from omegaconf import OmegaConf

default_setting = './configs/benchmark/default_setting.yaml'
strategy_setting = './configs/benchmark/entropy_sampling.yaml'
cfg = jupyter_parser(default_setting, strategy_setting)


# dataset 
from datasets import create_dataset
trainset, testset = create_dataset(
        normal_dataset  = cfg.DATASET.ID,
        anomaly_dataset = cfg.DATASET.OOD,
        datadir         = cfg.DATASET.datadir,
        img_size        = cfg.DATASET.img_size,
        mean            = cfg.DATASET.mean,
        std             = cfg.DATASET.std,
        aug_info        = cfg.DATASET.aug_info,
        anomaly_ratio   = cfg.DATASET.anomaly_ratio,
        total_size      = cfg.DATASET.total_size
    )   
from torch.utils.data import DataLoader 
trainloader = DataLoader(trainset, batch_size=32, shuffle=True)

# model 
from models.simclr import ResNetSimCLR
model = ResNetSimCLR(
                    modelname  = cfg.MODEL.modelname,
                    img_size   = cfg.DATASET.img_size,
                    pretrained = cfg.MODEL.pretrained,
                    out_dim    = cfg.MODEL.out_dim
                    )

# criterion 
criterion = __import__('criterions').__dict__[cfg.CRITERION.name](
    batch_size = cfg.DATASET.batch_size,
    **cfg.CRITERION.get('params',{})
    )

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Files already downloaded and verified
Using downloaded and verified file: /Volume/Data/SVHN/train_32x32.mat
Files already downloaded and verified
Using downloaded and verified file: /Volume/Data/SVHN/test_32x32.mat


In [2]:
from accelerate import Accelerator
accelerator = Accelerator() 
exp_name           = 'temp',
modelname          = cfg.MODEL.modelname
pretrained         = cfg.MODEL.pretrained
strategy           = cfg.AL.strategy
n_start            = cfg.AL.n_start
n_end              = cfg.AL.n_end
n_query            = cfg.AL.n_query
n_subset           = cfg.AL.n_subset
init_method        = cfg.AL.init.method
init_method_params = cfg.AL.init.get('params', {})
trainset           = trainset
validset           = testset
testset            = testset
criterion_name     = cfg.CRITERION.name
criterion_params   = cfg.CRITERION.get('params',{})
img_size           = cfg.DATASET.img_size
num_classes        = cfg.DATASET.num_classes
batch_size         = cfg.DATASET.batch_size
test_batch_size    = cfg.DATASET.test_batch_size
num_workers        = cfg.DATASET.num_workers
opt_name           = cfg.OPTIMIZER.opt_name
lr                 = cfg.OPTIMIZER.lr
opt_params         = cfg.OPTIMIZER.get('params',{})
epochs             = cfg.TRAIN.epochs
log_interval       = cfg.TRAIN.log_interval
use_wandb          = cfg.TRAIN.wandb.use
savedir            = 'temp'
seed               = cfg.DEFAULT.seed
accelerator        = accelerator
ckp_metric         = 'temp'
cfg                = cfg

In [11]:
cfg.AL.init.get('params',{})

{}

In [5]:
# inital sampling labeling
labeled_idx = create_labeled_index(
    method   = init_method,
    trainset = trainset,
    size     = n_start,
    seed     = seed,
    **init_method_params
)

# create model 
model = ResNetSimCLR(
                modelname  = cfg.MODEL.modelname,
                img_size   = cfg.DATASET.img_size,
                pretrained = cfg.MODEL.pretrained,
                out_dim    = cfg.MODEL.out_dim
                )

# create criterion 
criterion = __import__('criterions').__dict__[criterion_name](
                batch_size = batch_size,
                **criterion_params
                )

# select strategy    
strategy = create_query_strategy(
    strategy_name = strategy, 
    model         = model,
    dataset       = trainset, 
    labeled_idx   = labeled_idx, 
    n_query       = n_query, 
    batch_size    = batch_size, 
    num_workers   = num_workers,
    params        = cfg.AL.get('params', {})
)

# define train dataloader
trainloader = DataLoader(
    dataset     = trainset,
    batch_size  = batch_size,
    sampler     = SubsetRandomSampler(indices=np.where(labeled_idx==False)[0]),
    num_workers = num_workers
)

TypeError: create_labeled_index() argument after ** must be a mapping, not tuple